In [21]:
import torch
import torch.nn.functional as F

In [2]:

torch.manual_seed(1337)
B,T,C = 4,8,2

x = torch.randn(B,T,C)
# print(x)
x.shape

torch.Size([4, 8, 2])

In [3]:
x_bow = torch.zeros(B,T,C)

for b in range(B):
    for t in range(T):
        x_prev = x[b,:t+1]
        x_bow[b,t]=torch.mean(x_prev, 0)

In [4]:
print(x[1])
x_bow[1]

tensor([[ 1.3488, -0.1396],
        [ 0.2858,  0.9651],
        [-2.0371,  0.4931],
        [ 1.4870,  0.5910],
        [ 0.1260, -1.5627],
        [-1.1601, -0.3348],
        [ 0.4478, -0.8016],
        [ 1.5236,  2.5086]])


tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0.0020],
        [ 0.0712, -0.1128],
        [ 0.2527,  0.2149]])

In [5]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)

In [6]:
# x_bow2 = torch.zeros(B,T,C)

# for b in range(B):
#     x_prev = wei @ x[b] # (T, T) @ (T, C)
#     x_bow2[b] = x_prev

x_bow2 = wei @ x

In [10]:
x_bow2.shape

torch.Size([4, 8, 2])

In [8]:
x_bow2[1], x_bow[1]

(tensor([[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]]),
 tensor([[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]]))

In [16]:
torch.allclose(x_bow, x_bow2, atol=1e-06, rtol=1e-05)

True

In [34]:
tril = torch.tril(torch.ones(T,T))
wei = torch.ones(T,T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print(wei)
x_bow3 = wei @ x

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [35]:
torch.allclose(x_bow, x_bow3, atol=1e-06, rtol=1e-05)

True

In [42]:
ten = torch.tensor([[1,2],[3,4],[5,6]])
print(ten.shape)
ten = torch.stack([ten, ten])
print(ten.shape)

torch.Size([3, 2])
torch.Size([2, 3, 2])


## Cross entorpy test

In [43]:
logits = torch.randn((1, 4))
logits = torch.tensor([[0,0,0,1]], dtype=torch.float)
targets = torch.tensor([[0,0,1,0]], dtype=torch.float)
# targets = torch.randint(0, 4, (1,))
print(f"logits shape: {logits.shape} - {logits}")
print(f"Target shape: {targets.shape} - {targets}")
ret = F.cross_entropy(logits, targets)
print(ret)


logits shape: torch.Size([1, 4]) - tensor([[0., 0., 0., 1.]])
Target shape: torch.Size([1, 4]) - tensor([[0., 0., 1., 0.]])
tensor(1.7437)


In [81]:

logits_m = torch.tensor([[0.2,0.2,0.4,0.2]], dtype=torch.float)
idx_next = torch.multinomial(logits_m, num_samples=10, replacement=True)
print(idx_next)

tensor([[2, 2, 2, 1, 0, 2, 2, 1, 2, 1]])
